In [2]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import cv2
from tensorflow import keras
from tensorflow.keras.layers import Dense,Conv2D,Dropout,MaxPooling2D,Flatten
from tensorflow.keras import models,Sequential

In [3]:
def load_img(folder_path):
    img_name = os.listdir(folder_path)
    img = []
    i = 0
    for name in img_name:
        if i < 100:
            image = cv2.imread(os.path.join(folder_path,name))
            img.append(image)
            i+=1
    return np.array(img)

In [4]:
train_t = load_img('C://Users//pajag//OneDrive//Desktop//img apple & tomato//train//tomatoes')
train_a = load_img('C://Users//pajag//OneDrive//Desktop//img apple & tomato//train//apples')
test_t = load_img('C://Users//pajag//OneDrive//Desktop//img apple & tomato//test//tomatoes')
test_a = load_img('C://Users//pajag//OneDrive//Desktop//img apple & tomato//test//apples')

<ipython-input-3-08a29b18ba9b>:10: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return np.array(img)


In [5]:
train_a.shape,train_t.shape

((100,), (100,))

In [6]:
test_a.shape,test_t.shape

((54,), (43,))

In [7]:
img_size = (64,64)
def preprocess(arr):
    resize = []
    for img in arr:
        img = cv2.resize(img,img_size)
        img = img/255
        resize.append(img)
    return resize

In [8]:
train_a_preprocess = preprocess(train_a)
train_t_preprocess = preprocess(train_t)

In [9]:
test_a_preprocess = preprocess(test_a)
test_t_preprocess = preprocess(test_t)

In [10]:
import pandas as pd

In [11]:
train_a_df = pd.DataFrame({'images':train_a_preprocess,'label':'0'})
train_t_df = pd.DataFrame({'images':train_t_preprocess,'label':'1'})

In [12]:
test_a_df = pd.DataFrame({'images':test_a_preprocess,'label':'0'})
test_t_df = pd.DataFrame({'images':test_t_preprocess,'label':'1'})

In [13]:
data_train = pd.concat([train_a_df,train_t_df])

In [14]:
data_test = pd.concat([test_a_df,test_t_df])

In [15]:
data_train = data_train.iloc[np.random.permutation(len(data_train))]

In [16]:
data_test = data_test.iloc[np.random.permutation(len(data_test))]

In [17]:
data_train.head()

,images,label
92,"[[[0.4588235294117647, 0.47058823529411764, 0....",1
78,"[[[0.08235294117647059, 0.22745098039215686, 0...",1
13,"[[[1.0, 1.0, 1.0], [1.0, 1.0, 1.0], [1.0, 1.0,...",0
56,"[[[0.9529411764705882, 0.9725490196078431, 0.9...",0
84,"[[[1.0, 1.0, 1.0], [1.0, 1.0, 1.0], [1.0, 1.0,...",0


In [18]:
data_test.head()

,images,label
17,"[[[0.6392156862745098, 0.37254901960784315, 0....",0
25,"[[[1.0, 1.0, 1.0], [0.9607843137254902, 0.9607...",0
27,"[[[0.5098039215686274, 0.6705882352941176, 0.7...",0
16,"[[[0.24313725490196078, 0.40784313725490196, 0...",1
20,"[[[1.0, 1.0, 1.0], [1.0, 1.0, 1.0], [1.0, 1.0,...",0


In [22]:
model = models.Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(img_size[0], img_size[1], 3)))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

In [23]:
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [24]:
x = data_train['images']
y = data_train['label']

In [25]:
x = [tf.convert_to_tensor(img) for img in x]
x = tf.stack(x)
y = [tf.convert_to_tensor(val) for val in y]
y = tf.strings.to_number(y,out_type=tf.float32)

In [26]:
model.fit(x,y,batch_size=128,epochs=10)

Epoch 1/10
2/2 [==============================] - 1s 231ms/step - loss: 0.7010 - accuracy: 0.5350
Epoch 2/10
2/2 [==============================] - 1s 223ms/step - loss: 0.7349 - accuracy: 0.5200
Epoch 3/10
2/2 [==============================] - 1s 228ms/step - loss: 0.6809 - accuracy: 0.5150
Epoch 4/10
2/2 [==============================] - 1s 226ms/step - loss: 0.6511 - accuracy: 0.6400
Epoch 5/10
2/2 [==============================] - 1s 220ms/step - loss: 0.6497 - accuracy: 0.6650
Epoch 6/10
2/2 [==============================] - 1s 223ms/step - loss: 0.6276 - accuracy: 0.6750
Epoch 7/10
2/2 [==============================] - 1s 222ms/step - loss: 0.5997 - accuracy: 0.6800
Epoch 8/10
2/2 [==============================] - 1s 214ms/step - loss: 0.5963 - accuracy: 0.6800
Epoch 9/10
2/2 [==============================] - 1s 221ms/step - loss: 0.5923 - accuracy: 0.7000
Epoch 10/10
2/2 [==============================] - 1s 217ms/step - loss: 0.5616 - accuracy: 0.7250


In [27]:
def evaluate_model(data):
    test_images = [tf.convert_to_tensor(images) for images in data['images'].values]
    test_images = tf.stack(test_images)

    test_labels = tf.convert_to_tensor(data['label'].values)
    test_labels = tf.strings.to_number(test_labels, out_type=tf.float32)

    loss, accuracy = model.evaluate(test_images, test_labels)
    return accuracy

In [28]:
evaluate_model(data_test)

4/4 [==============================] - 0s 21ms/step - loss: 0.5834 - accuracy: 0.6907


0.6907216310501099

In [ ]:
def load_img(image_path):
    # Load the image using OpenCV
    image = cv2.imread(image_path)
    
    # Check if the image was loaded successfully
    if image is None:
        print(f"Failed to load image from path: {image_path}")
    else:
        # Display the loaded image
        cv2.imshow('Loaded Image', image)
        cv2.waitKey(0)
        cv2.destroyAllWindows()
    
    return image
